In [1]:
import json
from datasets import load_dataset

In [2]:
# Load the dataset
dataset = load_dataset("potsawee/speecheval-advanced-v1")["train"]
print("len(dataset):", len(dataset))

Repo card metadata block was not found. Setting CardData to empty.


len(dataset): 82


In [3]:
def read_judge_output_dir(judge_output_dir):
    winner_content_ref, winner_content_can = 0, 0
    winner_voice_ref, winner_voice_can = 0, 0
    error = 0
    for i in range(len(dataset)):
        x = dataset[i]
        conversation_id = x["id"]
        ref_model_type = x["position"]
        judge_output_path = f"{judge_output_dir}/{conversation_id}.txt"
        with open(judge_output_path) as f:
            y = json.load(f)
        response = y['response']
        # calculate winner ratio
        verdict = response.split("[Rankings]")[-1].strip("```").strip("json").strip("python").strip()
        parsed = json.loads(verdict)

        if int(parsed['content'][0]['rank']) == int(parsed['content'][1]['rank']):
            winner_content_ref += 0.5
            winner_content_can += 0.5
        else:
            yy = parsed['content'][0]
            if int(yy['rank']) == 1:
                if yy['model'] == 'model_1' and ref_model_type == 'model_1':
                    winner_content_ref += 1
                elif yy['model'] == 'model_1' and ref_model_type == 'model_2':
                    winner_content_can += 1
                elif yy['model'] == 'model_2' and ref_model_type == 'model_1':
                    winner_content_can += 1
                elif yy['model'] == 'model_2' and ref_model_type == 'model_2':
                    winner_content_ref += 1
                else:
                    raise Exception()
            elif int(yy['rank']) == 2:
                if yy['model'] == 'model_1' and ref_model_type == 'model_1':
                    winner_content_can += 1
                elif yy['model'] == 'model_1' and ref_model_type == 'model_2':
                    winner_content_ref += 1
                elif yy['model'] == 'model_2' and ref_model_type == 'model_1':
                    winner_content_ref += 1
                elif yy['model'] == 'model_2' and ref_model_type == 'model_2':
                    winner_content_can += 1
            else:
                error += 1
                continue

        if int(parsed['voice'][0]['rank']) == int(parsed['voice'][1]['rank']):
            winner_voice_ref += 0.5
            winner_voice_can += 0.5
        else:
            zz = parsed['voice'][0]
            if int(zz['rank']) == 1:
                if zz['model'] == 'model_1' and ref_model_type == 'model_1':
                    winner_voice_ref += 1
                elif zz['model'] == 'model_1' and ref_model_type == 'model_2':
                    winner_voice_can += 1
                elif zz['model'] == 'model_2' and ref_model_type == 'model_1':
                    winner_voice_can += 1
                elif zz['model'] == 'model_2' and ref_model_type == 'model_2':
                    winner_voice_ref += 1
                else:
                    raise Exception()
            elif int(zz['rank']) == 2:
                if zz['model'] == 'model_1' and ref_model_type == 'model_1':
                    winner_voice_can += 1
                elif zz['model'] == 'model_1' and ref_model_type == 'model_2':
                    winner_voice_ref += 1
                elif zz['model'] == 'model_2' and ref_model_type == 'model_1':
                    winner_voice_ref += 1
                elif zz['model'] == 'model_2' and ref_model_type == 'model_2':
                    winner_voice_can += 1
            else:
                error += 1
                continue
    print("[Content] Candidate winner percentage: {:.2f}%".format(winner_content_can / (winner_content_can + winner_content_ref) * 100))
    print("[Voice] Candidate winner percentage:   {:.2f}%".format(winner_voice_can / (winner_voice_can + winner_voice_ref) * 100))
    print("error:", error)

In [8]:
read_judge_output_dir("judge_outputs/advvoiceq1_v2/judge_gpt4o_prompt1.1_candidate_typ2kokoro")

[Content] Candidate winner percentage: 20.12%
[Voice] Candidate winner percentage:   12.20%
error: 0


In [9]:
read_judge_output_dir("judge_outputs/advvoiceq1_v2/judge_gpt4o_prompt1.1_candidate_divakokoro")


[Content] Candidate winner percentage: 34.15%
[Voice] Candidate winner percentage:   45.73%
error: 0


In [4]:
read_judge_output_dir("judge_outputs/advvoiceq1_v2/judge_gpt4o_prompt1.1_candidate_gemini2flashaudio")


[Content] Candidate winner percentage: 41.46%
[Voice] Candidate winner percentage:   21.34%
error: 0
